In [54]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
train_data = pd.read_csv('./training_data.csv')
test_data = pd.read_csv('./test_data.csv')

In [56]:
train_data.head()

,PatientID,Resp,PR Seq,RT Seq,VL-t0,CD4-t0
0,1,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCCCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAGCTAAAGCCAGGAA...,4.3,145
1,2,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.6,224
2,3,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGGTAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.2,1017
3,4,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,5.7,206
4,5,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAGTAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.5,572


In [57]:
print(train_data['PR Seq'].isnull().sum())
print(train_data['RT Seq'].isnull().sum())
print(test_data['PR Seq'].isnull().sum())
print(test_data['RT Seq'].isnull().sum())

80
0
0
0


In [58]:
# drop 80 rows because of NaNs
train_data.dropna(subset=['PR Seq'], inplace=True)

In [59]:

Y = train_data['Resp'].values
X = train_data.drop(['PatientID', 'Resp'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [60]:
x_train.tail()

,PR Seq,RT Seq,VL-t0,CD4-t0
106,CCTCAAATCACTCTTTGGCAGCGACCCATTGTTACAGTAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,5.4,319
270,CCTCAGATCACTCTTTGGCAACGACCCCGCGTCACAGTAARGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,5.4,56
860,CCTCARATCACTCTTTGGCAACGACCCCTCGTCACAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,4.2,496
435,CCTCAGATCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAG...,CCCATAAGTCCTATTGAAACTGTACCAGTAAAATTRAAGCCAGGAA...,3.9,327
102,CCTCAAATCACTCTTTGGCAAMGACCCCTCGTCACAATAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,4.7,546


In [61]:
catboost = CatBoostClassifier(cat_features=['PR Seq', 'RT Seq'], verbose=False)
catboost.fit(x_train, y_train)

In [62]:
# on test partition of train data
y_pred = catboost.predict(x_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8043478260869565

In [63]:
# apply it to the actual test set
catboost.fit(X, Y)

Yt = test_data['Resp'].values
Xt = test_data.drop(['PatientID', 'Resp'], axis=1)
yt_pred = catboost.predict(Xt)


In [74]:
# instead of 0s and 1s there is an H in the response
test_data[test_data['Resp']!='H']

,PatientID,Resp,PR Seq,RT Seq,VL-t0,CD4-t0


In [75]:
submissions = pd.DataFrame({
    'PatientID': np.arange(1,yt_pred.shape[0]+1),
    'ResponderStatus': yt_pred
})
submissions.to_csv('submission.csv', index=False)